In [2]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import os
import re
import logging
import json
from typing import Optional, Tuple, List
from pathlib import Path
from tqdm.notebook import tqdm
import urllib3
from concurrent.futures import ThreadPoolExecutor, as_completed
from queue import Queue
import threading

# --- Setup Logging ---
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('herbalmdb_scraper.log', encoding='utf-8'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# --- Load Configuration ---
DEFAULT_CONFIG = {
    "OVERALL_START_ID": 22668141,
    "OVERALL_END_ID": 23478140,
    "BATCH_SIZE": 10000,
    "SAVE_DIRECTORY": "HerbalMDB_Class",
    "BASE_URL": "https://cbcb.cdutcm.edu.cn/smdb/secondary_metabolites_detail/",  
    "REQUEST_TIMEOUT": 30,
    "SLEEP_INTERVAL": 0.5,
    "VERIFY_SSL": False,
    "MAX_CONCURRENT_REQUESTS": 20,
    "MAX_RETRIES": 10,
    "BATCH_RETRY_ATTEMPTS": 3  # 新增：批次重试次数
}

def load_config(config_path: str = "config.json") -> dict:
    """Load configuration from a JSON file or return default configuration."""
    try:
        if os.path.exists(config_path):
            with open(config_path, 'r', encoding='utf-8') as f:
                config = json.load(f)
                logger.info(f"Loaded configuration from {config_path}")
                return {**DEFAULT_CONFIG, **config}
        else:
            logger.info("No config file found, using default configuration")
            return DEFAULT_CONFIG
    except Exception as e:
        logger.error(f"Failed to load config file {config_path}: {e}")
        return DEFAULT_CONFIG

CONFIG = load_config()

# --- Global Configuration ---
OVERALL_START_ID = CONFIG["OVERALL_START_ID"]
OVERALL_END_ID = CONFIG["OVERALL_END_ID"]
BATCH_SIZE = CONFIG["BATCH_SIZE"]
SAVE_DIRECTORY = CONFIG["SAVE_DIRECTORY"]
BASE_URL = CONFIG["BASE_URL"]
REQUEST_TIMEOUT = CONFIG["REQUEST_TIMEOUT"]
SLEEP_INTERVAL = CONFIG["SLEEP_INTERVAL"]
MAX_CONCURRENT_REQUESTS = CONFIG["MAX_CONCURRENT_REQUESTS"]
MAX_RETRIES = CONFIG["MAX_RETRIES"]
BATCH_RETRY_ATTEMPTS = CONFIG["BATCH_RETRY_ATTEMPTS"]

if not CONFIG["VERIFY_SSL"]:
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

def get_last_processed_id(filename: str) -> Optional[int]:
    """Check the CSV file to find the last ID that was attempted (regardless of success/failure)."""
    if not os.path.exists(filename):
        logger.info(f"File {filename} does not exist, starting from scratch")
        return None
    try:
        with open(filename, 'r', encoding='utf-8') as csvfile:
            reader = csv.DictReader(csvfile)
            last_id = None
            for row in reader:
                if row.get('ID'):
                    try:
                        last_id = int(row['ID'])
                    except ValueError:
                        continue
            logger.info(f"Last processed ID in {filename}: {last_id}")
            return last_id
    except (IOError, csv.Error, KeyError) as e:
        logger.warning(f"Failed to read last processed ID from {filename}: {e}")
        return None

def verify_batch_completeness(start_id: int, end_id: int, filename: str) -> Tuple[bool, List[int]]:
    """Verify that all IDs in the range have been processed and return missing IDs."""
    if not os.path.exists(filename):
        return False, list(range(start_id, end_id + 1))
    
    processed_ids = set()
    try:
        with open(filename, 'r', encoding='utf-8') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                if row.get('ID'):
                    try:
                        processed_ids.add(int(row['ID']))
                    except ValueError:
                        continue
    except Exception as e:
        logger.error(f"Error verifying completeness: {e}")
        return False, list(range(start_id, end_id + 1))
    
    expected_ids = set(range(start_id, end_id + 1))
    missing_ids = sorted(list(expected_ids - processed_ids))
    
    if missing_ids:
        logger.warning(f"Missing IDs in batch {start_id}-{end_id}: {len(missing_ids)} IDs missing")
        return False, missing_ids
    
    logger.info(f"Batch {start_id}-{end_id} is complete with all {len(expected_ids)} IDs processed")
    return True, []

def get_info_by_label(soup: BeautifulSoup, label_text: str) -> str:
    """Extract information by finding a div with the specified label text and retrieving the text from its next sibling div."""
    try:
        label_pattern = re.compile(r'\s*' + re.escape(label_text) + r'\s*', re.IGNORECASE)
        text_node = soup.find(string=label_pattern)
        if text_node:
            label_div = text_node.find_parent('div')
            if label_div:
                value_div = label_div.find_next_sibling('div')
                if value_div:
                    return value_div.get_text(strip=True)
    except Exception as e:
        logger.debug(f"Error extracting {label_text}: {e}")
    return ''

def extract_data_from_soup(soup: BeautifulSoup) -> Tuple[str, str, str, str, str]:
    """Extract all required data points from the BeautifulSoup object."""
    try:
        smiles_tag = soup.find('span', id='smiles-text')
        smiles = smiles_tag.get_text(strip=True) if smiles_tag else ''
        logger.debug(f"Extracted SMILES: {smiles}")

        common_name = ''
        cn_label_tag = soup.find('div', string=re.compile(r'\s*Common Name\s*', re.IGNORECASE))
        if cn_label_tag:
            cn_value_tag = cn_label_tag.find_next_sibling('div')
            if cn_value_tag and cn_value_tag.has_attr('title'):
                common_name = cn_value_tag['title']
        logger.debug(f"Extracted Common Name: {common_name}")

        class_val = get_info_by_label(soup, 'Class')
        superclass_val = get_info_by_label(soup, 'Superclass')
        pathway_val = get_info_by_label(soup, 'Pathway')
        logger.debug(f"Extracted Class: {class_val}, Superclass: {superclass_val}, Pathway: {pathway_val}")

        return smiles, common_name, class_val, superclass_val, pathway_val
    except Exception as e:
        logger.error(f"Error extracting data from soup: {e}")
        return '', '', '', '', ''

def fetch_page(id: int, retries: int = 0) -> Tuple[int, Optional[BeautifulSoup], Optional[str]]:
    """Fetch a single page and return the ID, parsed soup, and any error message."""
    url = f"{BASE_URL}{id}/"
    try:
        response = requests.get(url, timeout=REQUEST_TIMEOUT, verify=CONFIG["VERIFY_SSL"])
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        return id, soup, None
    except requests.exceptions.RequestException as e:
        if retries < MAX_RETRIES:
            wait_time = (retries + 1) * 2  # Exponential backoff
            logger.warning(f"ID {id}: Fetch failed ({url}). Retrying in {wait_time}s... (Attempt {retries+1}/{MAX_RETRIES})")
            time.sleep(wait_time)
            return fetch_page(id, retries + 1)
        else:
            logger.error(f"ID {id}: Fetch failed after {MAX_RETRIES} retries ({url}). Error: {e}")
            return id, None, str(e)

def process_ids_batch(id_list: List[int], file_lock: threading.Lock, filename: str) -> Tuple[int, int]:
    """Process a batch of IDs and return success/error counts."""
    batch_rows = []
    successful_count = 0
    error_count = 0

    def write_to_csv(rows: List[list]):
        """Thread-safe CSV writing."""
        if not rows:
            return
        try:
            with file_lock:
                with open(filename, 'a', newline='', encoding='utf-8') as csvfile:
                    csv_writer = csv.writer(csvfile)
                    for row in rows:
                        csv_writer.writerow(row)
                logger.debug(f"Successfully wrote {len(rows)} rows to {filename}")
        except IOError as e:
            logger.error(f"Failed to write to {filename}: {e}")
            raise

    with ThreadPoolExecutor(max_workers=MAX_CONCURRENT_REQUESTS) as executor:
        future_to_id = {executor.submit(fetch_page, i): i for i in id_list}
        
        for future in tqdm(as_completed(future_to_id), total=len(id_list), 
                          desc=f"Processing {len(id_list)} IDs", unit=" ID", leave=False):
            current_id = future_to_id[future]
            
            try:
                result_id, soup, error = future.result()
                
                if error:
                    batch_rows.append([result_id, 'FETCH_ERROR', str(error), '', '', ''])
                    error_count += 1
                    logger.debug(f"ID {result_id}: Fetch error recorded")
                else:
                    try:
                        data = extract_data_from_soup(soup)
                        batch_rows.append([result_id] + list(data))
                        successful_count += 1
                        logger.debug(f"ID {result_id}: Successfully extracted data")
                    except Exception as e:
                        logger.error(f"ID {result_id}: Parsing failed. Error: {e}")
                        batch_rows.append([result_id, 'PARSE_ERROR', str(e), '', '', ''])
                        error_count += 1
                        
            except Exception as e:
                logger.error(f"ID {current_id}: Unexpected error in future processing: {e}")
                batch_rows.append([current_id, 'UNEXPECTED_ERROR', str(e), '', '', ''])
                error_count += 1

            # Write in smaller batches to avoid memory issues
            if len(batch_rows) >= 50:
                try:
                    write_to_csv(batch_rows)
                    batch_rows = []
                except Exception as e:
                    logger.error(f"Critical error writing to CSV: {e}")
                    break

            # Rate limiting
            if SLEEP_INTERVAL > 0:
                time.sleep(SLEEP_INTERVAL / MAX_CONCURRENT_REQUESTS)

    # Write any remaining rows
    if batch_rows:
        try:
            write_to_csv(batch_rows)
        except Exception as e:
            logger.error(f"Failed to write final batch: {e}")

    return successful_count, error_count

def scrape_batch(start_id: int, end_id: int, filename: str):
    """Scrape data for the given ID range and save it to a CSV file with automatic retry for missing IDs."""
    logger.info(f"Starting batch: ID {start_id} -> {end_id}")
    logger.info(f"Data will be saved to: {filename}")

    # Check if batch is already complete
    is_complete, missing_ids = verify_batch_completeness(start_id, end_id, filename)
    if is_complete:
        logger.info(f"Batch {start_id}-{end_id} already completed. Skipping.")
        return

    file_lock = threading.Lock()

    # Ensure CSV file exists with header
    file_exists = os.path.exists(filename)
    if not file_exists:
        with file_lock:
            try:
                with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
                    csv_writer = csv.writer(csvfile)
                    csv_writer.writerow(['ID', 'SMILES', 'Common Name', 'Class', 'Superclass', 'Pathway'])
                    logger.info(f"Created new CSV file with header: {filename}")
            except IOError as e:
                logger.error(f"Failed to create CSV file {filename}: {e}")
                return

    # Determine which IDs to process initially
    if missing_ids:
        logger.info(f"Processing {len(missing_ids)} missing IDs in batch {start_id}-{end_id}")
        id_range = missing_ids
    else:
        # If file doesn't exist or verification failed, process all IDs
        last_processed_id = get_last_processed_id(filename)
        current_start_id = start_id
        if last_processed_id and last_processed_id >= start_id:
            current_start_id = last_processed_id + 1
            logger.info(f"Resuming from ID {current_start_id}")

        if current_start_id > end_id:
            logger.info(f"Batch {start_id}-{end_id} already completed. Skipping.")
            return

        id_range = list(range(current_start_id, end_id + 1))

    # Initial processing
    logger.info(f"Initial processing of {len(id_range)} IDs")
    successful_count, error_count = process_ids_batch(id_range, file_lock, filename)
    logger.info(f"Initial processing completed. Success: {successful_count}, Errors: {error_count}")

    # Automatic retry mechanism for missing IDs
    retry_count = 0
    while retry_count < BATCH_RETRY_ATTEMPTS:
        is_complete, remaining_missing = verify_batch_completeness(start_id, end_id, filename)
        
        if is_complete:
            logger.info(f"✅ Batch {start_id}-{end_id} is now complete!")
            break
            
        if remaining_missing:
            retry_count += 1
            logger.warning(f"🔄 Batch {start_id}-{end_id} has {len(remaining_missing)} missing IDs. "
                          f"Attempting retry {retry_count}/{BATCH_RETRY_ATTEMPTS}")
            
            # 重新处理遗漏的ID
            retry_successful, retry_errors = process_ids_batch(remaining_missing, file_lock, filename)
            logger.info(f"Retry {retry_count} completed. Success: {retry_successful}, Errors: {retry_errors}")
            
            if retry_count < BATCH_RETRY_ATTEMPTS:
                logger.info(f"Waiting 10 seconds before next retry attempt...")
                time.sleep(10)  # 等待10秒再重试
        else:
            break
    
    # Final status
    final_complete, final_missing = verify_batch_completeness(start_id, end_id, filename)
    if final_complete:
        logger.info(f"🎉 Batch {start_id}-{end_id} successfully completed!")
    else:
        logger.error(f"❌ Failed to complete batch {start_id}-{end_id} after {BATCH_RETRY_ATTEMPTS} retry attempts. "
                    f"{len(final_missing)} IDs still missing: {final_missing[:10]}{'...' if len(final_missing) > 10 else ''}")

def main():
    """Main function to coordinate the entire scraping process."""
    try:
        os.makedirs(SAVE_DIRECTORY, exist_ok=True)
        logger.info(f"Data save directory confirmed: {SAVE_DIRECTORY}")
    except OSError as e:
        logger.error(f"Unable to create directory {SAVE_DIRECTORY}. Check permissions. Error: {e}")
        return

    total_batches = (OVERALL_END_ID - OVERALL_START_ID + BATCH_SIZE) // BATCH_SIZE
    completed_batches = 0

    for batch_start in range(OVERALL_START_ID, OVERALL_END_ID + 1, BATCH_SIZE):
        batch_end = min(batch_start + BATCH_SIZE - 1, OVERALL_END_ID)
        output_filename = os.path.join(SAVE_DIRECTORY, f"HerbalMDB_{batch_start}-{batch_end}.csv")

        logger.info("\n" + "="*80)
        logger.info(f"Processing batch {completed_batches + 1}/{total_batches}: IDs {batch_start} to {batch_end}")
        logger.info("="*80)

        try:
            scrape_batch(batch_start, batch_end, output_filename)
            completed_batches += 1
        except Exception as e:
            logger.error(f"Critical error in batch {batch_start}-{batch_end}: {e}")
            logger.info("Continuing with next batch...")

        if batch_end < OVERALL_END_ID:
            logger.info(f"Batch {batch_start}-{batch_end} processed. Next batch starts at {batch_end + 1}.")

    logger.info("\n" + "*"*80)
    logger.info(f"All specified ID ranges processed. Completed {completed_batches}/{total_batches} batches.")
    logger.info("*"*80)

if __name__ == "__main__":
    main()


2025-07-19 03:30:00,677 - INFO - No config file found, using default configuration
2025-07-19 03:30:00,680 - INFO - Data save directory confirmed: HerbalMDB_Class
2025-07-19 03:30:00,680 - INFO - 
2025-07-19 03:30:00,680 - INFO - Processing batch 1/81: IDs 22668141 to 22678140
2025-07-19 03:30:00,681 - INFO - ================================================================================
2025-07-19 03:30:00,681 - INFO - Starting batch: ID 22668141 -> 22678140
2025-07-19 03:30:00,681 - INFO - Data will be saved to: HerbalMDB_Class\HerbalMDB_22668141-22678140.csv
2025-07-19 03:30:00,682 - INFO - Created new CSV file with header: HerbalMDB_Class\HerbalMDB_22668141-22678140.csv
2025-07-19 03:30:00,683 - INFO - Processing 10000 missing IDs in batch 22668141-22678140
2025-07-19 03:30:00,684 - INFO - Initial processing of 10000 IDs


Processing 10000 IDs:   0%|          | 0/10000 [00:00<?, ? ID/s]

2025-07-19 03:38:17,509 - INFO - Initial processing completed. Success: 10000, Errors: 0
2025-07-19 03:38:17,540 - INFO - Batch 22668141-22678140 is complete with all 10000 IDs processed
2025-07-19 03:38:17,541 - INFO - ✅ Batch 22668141-22678140 is now complete!
2025-07-19 03:38:17,561 - INFO - Batch 22668141-22678140 is complete with all 10000 IDs processed
2025-07-19 03:38:17,562 - INFO - 🎉 Batch 22668141-22678140 successfully completed!
2025-07-19 03:38:17,564 - INFO - Batch 22668141-22678140 processed. Next batch starts at 22678141.
2025-07-19 03:38:17,565 - INFO - 
2025-07-19 03:38:17,565 - INFO - Processing batch 2/81: IDs 22678141 to 22688140
2025-07-19 03:38:17,566 - INFO - ================================================================================
2025-07-19 03:38:17,566 - INFO - Starting batch: ID 22678141 -> 22688140
2025-07-19 03:38:17,567 - INFO - Data will be saved to: HerbalMDB_Class\HerbalMDB_22678141-22688140.csv
2025-07-19 03:38:17,568 - INFO - Created new CSV fi

Processing 10000 IDs:   0%|          | 0/10000 [00:00<?, ? ID/s]

2025-07-19 03:39:07,902 - WARNING - ID 22678359: Fetch failed (https://cbcb.cdutcm.edu.cn/smdb/secondary_metabolites_detail/22678359/). Retrying in 2s... (Attempt 1/10)
2025-07-19 03:41:37,459 - WARNING - ID 22680709: Fetch failed (https://cbcb.cdutcm.edu.cn/smdb/secondary_metabolites_detail/22680709/). Retrying in 2s... (Attempt 1/10)
2025-07-19 03:47:57,209 - INFO - Initial processing completed. Success: 10000, Errors: 0
2025-07-19 03:47:57,240 - INFO - Batch 22678141-22688140 is complete with all 10000 IDs processed
2025-07-19 03:47:57,241 - INFO - ✅ Batch 22678141-22688140 is now complete!
2025-07-19 03:47:57,264 - INFO - Batch 22678141-22688140 is complete with all 10000 IDs processed
2025-07-19 03:47:57,264 - INFO - 🎉 Batch 22678141-22688140 successfully completed!
2025-07-19 03:47:57,265 - INFO - Batch 22678141-22688140 processed. Next batch starts at 22688141.
2025-07-19 03:47:57,265 - INFO - 
2025-07-19 03:47:57,266 - INFO - Processing batch 3/81: IDs 22688141 to 22698140
2025

...

2025-07-19 16:17:25,401 - WARNING - ID 23450476: Fetch failed (https://cbcb.cdutcm.edu.cn/smdb/secondary_metabolites_detail/23450476/). Retrying in 2s... (Attempt 1/10)
2025-07-19 16:17:25,417 - WARNING - ID 23450475: Fetch failed (https://cbcb.cdutcm.edu.cn/smdb/secondary_metabolites_detail/23450475/). Retrying in 2s... (Attempt 1/10)
2025-07-19 16:17:25,419 - WARNING - ID 23450478: Fetch failed (https://cbcb.cdutcm.edu.cn/smdb/secondary_metabolites_detail/23450478/). Retrying in 2s... (Attempt 1/10)
2025-07-19 16:17:25,432 - WARNING - ID 23450479: Fetch failed (https://cbcb.cdutcm.edu.cn/smdb/secondary_metabolites_detail/23450479/). Retrying in 2s... (Attempt 1/10)
2025-07-19 16:17:25,465 - WARNING - ID 23450477: Fetch failed (https://cbcb.cdutcm.edu.cn/smdb/secondary_metabolites_detail/23450477/). Retrying in 2s... (Attempt 1/10)
2025-07-19 16:24:28,811 - INFO - Initial processing completed. Success: 10000, Errors: 0
2025-07-19 16:24:28,842 - INFO - Batch 23448141-23458140 is comple

Processing 10000 IDs:   0%|          | 0/10000 [00:00<?, ? ID/s]

2025-07-19 16:27:21,696 - WARNING - ID 23460195: Fetch failed (https://cbcb.cdutcm.edu.cn/smdb/secondary_metabolites_detail/23460195/). Retrying in 2s... (Attempt 1/10)
2025-07-19 16:34:15,801 - INFO - Initial processing completed. Success: 10000, Errors: 0
2025-07-19 16:34:15,832 - INFO - Batch 23458141-23468140 is complete with all 10000 IDs processed
2025-07-19 16:34:15,832 - INFO - ✅ Batch 23458141-23468140 is now complete!
2025-07-19 16:34:15,855 - INFO - Batch 23458141-23468140 is complete with all 10000 IDs processed
2025-07-19 16:34:15,857 - INFO - 🎉 Batch 23458141-23468140 successfully completed!
2025-07-19 16:34:15,857 - INFO - Batch 23458141-23468140 processed. Next batch starts at 23468141.
2025-07-19 16:34:15,858 - INFO - 
2025-07-19 16:34:15,858 - INFO - Processing batch 81/81: IDs 23468141 to 23478140
2025-07-19 16:34:15,859 - INFO - ================================================================================
2025-07-19 16:34:15,860 - INFO - Starting batch: ID 234681

Processing 10000 IDs:   0%|          | 0/10000 [00:00<?, ? ID/s]

2025-07-19 16:35:10,242 - WARNING - ID 23468442: Fetch failed (https://cbcb.cdutcm.edu.cn/smdb/secondary_metabolites_detail/23468442/). Retrying in 2s... (Attempt 1/10)
2025-07-19 16:42:48,726 - WARNING - ID 23476432: Fetch failed (https://cbcb.cdutcm.edu.cn/smdb/secondary_metabolites_detail/23476432/). Retrying in 2s... (Attempt 1/10)
2025-07-19 16:44:05,825 - INFO - Initial processing completed. Success: 10000, Errors: 0
2025-07-19 16:44:05,853 - INFO - Batch 23468141-23478140 is complete with all 10000 IDs processed
2025-07-19 16:44:05,858 - INFO - ✅ Batch 23468141-23478140 is now complete!
2025-07-19 16:44:05,880 - INFO - Batch 23468141-23478140 is complete with all 10000 IDs processed
2025-07-19 16:44:05,881 - INFO - 🎉 Batch 23468141-23478140 successfully completed!
2025-07-19 16:44:05,883 - INFO - 
********************************************************************************
2025-07-19 16:44:05,883 - INFO - All specified ID ranges processed. Completed 81/81 batches.
2025-07-19